In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
#Create the matrix of process, limit, retention cost and penalty cost
df = pd.DataFrame({"Task": [1,2,3,4,5], "process_time": [10,8,6,7,4], "limit_time": [15,20,10,30,12], "retention_cost_per_unit": [3,2,5,4,6], "penalty_cost_per_unit": [10,22,10,8,15]})
df.set_index("Task", inplace=True)

In [3]:
df

,process_time,limit_time,retention_cost_per_unit,penalty_cost_per_unit
Task,,,,
1,10,15,3,10
2,8,20,2,22
3,6,10,5,10
4,7,30,4,8
5,4,12,6,15


In [4]:
def fitness(chromosome,df):
    process_time = 0
    retention_cost = 0
    penalty_cost = 0
    cost = 0
    for task in chromosome:
        process_time += df["process_time"][task]


        if process_time > df["limit_time"][task]:
            penalty_cost += df["penalty_cost_per_unit"][task] * (process_time - df["limit_time"][task])
        else:
            retention_cost += df["retention_cost_per_unit"][task] * (df["limit_time"][task] - process_time)
        cost = retention_cost + penalty_cost
    return -cost

In [5]:
# Se crea la clase de agente, cuenta con sus cromosomas, peso y recompensa
class Agent():
    def __init__(self, num_genes=5):
        self.chromosome = []
        self.num_genes = num_genes
        self.generate_valid_chromosome()
        self.fitness = fitness(self.chromosome,df)

    # Se crea la función de mutar el cual elige un gen al azar y lo cambia
    def mutate(self):
        gene = random.randint(0, self.num_genes - 1)
        original_gene_value = self.chromosome[gene]

        # Se muta el gen con un nuevo valor
        self.chromosome[gene] = random.choice([1, 2, 3, 4, 5])

        # Validar si hay duplicados
        while self.chromosome.count(self.chromosome[gene]) > 1:
            # Si hay duplicados, se vuelve a cambiar el gen mutado por uno no repetido
            self.chromosome[gene] = random.choice([x for x in [1, 2, 3, 4, 5] if x not in self.chromosome or x == original_gene_value])
        self.fitness = fitness(self.chromosome,df)

    def generate_valid_chromosome(self):
        while len(set(self.chromosome)) != self.num_genes:
            self.chromosome = [random.choice([1, 2, 3, 4, 5]) for _ in range(self.num_genes)]



In [50]:
#Se toma la mitad de la población con mejor fitness
def selection(population):
    sorted_population = sorted(population, key=lambda x: x.fitness, reverse=True)
    return sorted_population[:len(population)//2]

#Se realiza el cruce de los cromosomas de los agentes
def crossover(population):
    offspring = []

    # Convert the elements list to a numpy array
    arpopulation = np.array(population)

    # Shuffle the elements using numpy's shuffle
    np.random.shuffle(arpopulation)

    # Check if the number of elements is even
    if len(arpopulation) % 2 != 0:
        raise ValueError("The number of elements must be even to form pairs.")

    # Reshape the array into pairs
    arparents = arpopulation.reshape(-1, 2)

    # Convert the numpy array of pairs back to a list of lists
    parents = arparents.tolist()

    for parent in parents:
        parent1 = parent[0]
        parent2 = parent[1]
        child1 = Agent()
        child2 = Agent()
        child3 = Agent()
        child4 = Agent()
        child1.chromosome = parent1.chromosome[:3] + parent2.chromosome[3:]
        child2.chromosome = parent2.chromosome[:3] + parent1.chromosome[3:]
        child3.chromosome = parent2.chromosome[3:] + parent2.chromosome[:3]
        child4.chromosome = parent1.chromosome[3:] + parent1.chromosome[:3]
        childs = [child1, child2, child3, child4]

        # Se toman solo los dos mejores hijos de cada cruce
        mutated_childs = mutation(childs)
        #Eliminar los hijos que no cumplen con la condición de tener la secuencia de 1 a 5, en la lista de mutated_childs
        mutated_childs = [child for child in mutated_childs if len(set(child.chromosome)) == 5]
        mutated_childs = sorted(mutated_childs, key=lambda x: x.fitness)
        mutated_childs = mutated_childs[-2:]
        offspring.extend(mutated_childs)



    return offspring
# Se realiza la probabilidad de mutación
def mutation(population):
    for entity in population:
        if random.random() < 0.1: #Probabilidad de mutacion
            entity.mutate()
        entity.fitness = fitness(entity.chromosome, df)
    return population

In [97]:
# Se inicializa la población

population_size = 60
population = [Agent() for _ in range(population_size)]
counter = 0
past_agent = 0
epoch = 0
#Número de epocas

while epoch < 25:
    epoch += 1
    purged_population = selection(population)
    babies = crossover(purged_population)
    while len(babies) < len(population)//2:
        babies.append(crossover(purged_population))
    babies = babies[:len(population)//2]
    purged_population.extend(babies)
    population = purged_population
    #Look for the best agent
    best_agent = max(population, key=lambda x: x.fitness)
    if best_agent.fitness == past_agent:
        counter += 1
    else:
        counter = 0
    past_agent = best_agent.fitness
    print("Epoca:"+ str(epoch))
    print("Este es el fitness mejor agente: " + str(best_agent.fitness))
    print("Este es el cromosoma mejor agente: " + str(best_agent.chromosome))
    #Save the best agent of all time

Epoca:1
Este es el fitness mejor agente: -222
Este es el cromosoma mejor agente: [5, 3, 2, 1, 4]
Epoca:2
Este es el fitness mejor agente: -222
Este es el cromosoma mejor agente: [5, 3, 2, 1, 4]
Epoca:3
Este es el fitness mejor agente: -222
Este es el cromosoma mejor agente: [5, 3, 2, 1, 4]
Epoca:4
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:5
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:6
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:7
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:8
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:9
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:10
Este es el fitness mejor agente: -206
Este es el cromosoma mejor agente: [3, 5, 2, 1, 4]
Epoca:11
Este es el fitness m